In [55]:
import os
import requests
from bs4 import BeautifulSoup
import google.generativeai as gen
from kaggle_secrets import UserSecretsClient

# 1) Kaggle Secrets থেকে GOOGLE_API_KEY আনছি
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

if not GOOGLE_API_KEY:
    raise RuntimeError("GOOGLE_API_KEY secret set করা নাই – Kaggle Secrets এ add করে নাও।")

# 2) Gemini configure
gen.configure(api_key=GOOGLE_API_KEY)

# 3) **এখানে model hardcode করলাম** – আর auto select নয়
MODEL_NAME = "gemini-2.5-flash"      # lite ব্যবহার করতে চাইলে: "gemini-2.5-flash-lite"
model = gen.GenerativeModel(MODEL_NAME)

print("🤖 Using Gemini model:", MODEL_NAME)


🤖 Using Gemini model: gemini-2.5-flash


In [56]:
def fetch_article_from_url(url: str):
    """Simple scraper: সব <p> ট্যাগ join করে article বানাচ্ছি."""
    resp = requests.get(url, timeout=20)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")

    title = soup.title.string.strip() if soup.title else "Unknown Title"

    paragraphs = []
    for p in soup.find_all("p"):
        text = p.get_text(" ", strip=True)
        if len(text) > 40:   # খুব ছোট noise বাদ
            paragraphs.append(text)

    article_text = "\n".join(paragraphs)
    return title, article_text


In [57]:
def summarize_with_gemini(article_text: str, url: str = None, language: str = "bn") -> str:
    if not article_text or len(article_text.strip()) < 200:
        return "Article text is too short or could not be extracted."

    # input token কম রাখার জন্য প্রায় ~4000 character এ কেটে নিচ্ছি
    max_chars = 4000
    if len(article_text) > max_chars:
        article_text = article_text[:max_chars]

    if language == "bn":
        instruction = (
            "তুমি একজন AI News Summarizer এজেন্ট। "
            "নিচের নিউজ আর্টিকেলটি পড়ে ৫–৭টি পয়েন্টে সহজ ভাষায় বাংলা সারাংশ লিখো। "
            "মূল ঘটনা, সময়, স্থান, কারণ আর প্রভাব যেন পরিষ্কার থাকে।"
        )
    else:
        instruction = (
            "You are an AI news summarization agent. "
            "Read the article and summarize it in 5–7 clear bullet points in English, "
            "covering who, what, when, where, why, and key outcomes."
        )

    prompt = f"""{instruction}

URL: {url or "N/A"}

Article:
\"\"\"
{article_text}
\"\"\""""

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Gemini error: {e}"


In [58]:
def summarize_news_url(url: str, language: str = "bn"):
    print("🌐 Fetching article...")
    title, article_text = fetch_article_from_url(url)
    print("📝 Extracted article length:", len(article_text))

    print("🤖 Sending to Gemini 2.5-flash...")
    summary = summarize_with_gemini(article_text, url=url, language=language)

    return title, summary


In [59]:
url = "https://www.bbc.com/news/articles/c891jp9j79do"  # এখানে যেকোনো news link দাও

title, summary = summarize_news_url(url, language="bn")  # "en" দিলে English summary

print("\n===== 📰 ARTICLE TITLE =====")
print(title)

print("\n===== ✨ GENERATED SUMMARY =====\n")
print(summary)


🌐 Fetching article...
📝 Extracted article length: 4722
🤖 Sending to Gemini 2.5-flash...

===== 📰 ARTICLE TITLE =====
Trump says he will sue BBC for at least $1bn over Panorama edit

===== ✨ GENERATED SUMMARY =====

একজন AI News Summarizer এজেন্ট হিসেবে, বিবিসি নিউজ আর্টিকেলটির সারসংক্ষেপ নিচে ৫–৭টি পয়েন্টে সহজ বাংলায় দেওয়া হলো:

*   **১. মূল ঘটনা ও সময়:** মার্কিন যুক্তরাষ্ট্রের সাবেক প্রেসিডেন্ট ডোনাল্ড ট্রাম্প বিবিসি-র বিরুদ্ধে ১ থেকে ৫ বিলিয়ন ডলার ক্ষতিপূরণের মামলা করার ঘোষণা দিয়েছেন। বিবিসি তাদের 'প্যানোরামা' ডকুমেন্টারিতে (২০২৪ সালের অক্টোবরে প্রচারিত) ট্রাম্পের ২০২১ সালের ৬ জানুয়ারির ভাষণ বিকৃতভাবে সম্পাদনা করার কারণে এই মামলার হুমকি দেওয়া হয়েছে।

*   **২. কারণ:** বিবিসি-র সম্পাদনার ফলে ভুলভাবে বোঝানো হয়েছে যে, ট্রাম্প ক্যাপিটল হিলে সরাসরি সহিংস পদক্ষেপের জন্য উসকানি দিয়েছিলেন। বিবিসি স্বীকার করেছে যে তাদের সম্পাদনা 'অনিচ্ছাকৃতভাবে' এই ভুল ধারণা তৈরি করেছে, যেখানে ভাষণের বিভিন্ন অংশ জুড়ে দিয়ে একটি ধারাবাহিক বক্তব্য দেখানো হয়েছে।

*   **৩. বিবিসির প্রতিক্রিয়া:** বিবিসি এ

In [60]:
from urllib.parse import urlparse

def validate_url(url: str):
    url = (url or "").strip()
    if not url:
        return False, "URL ফাঁকা দেওয়া হয়েছে।"

    if not (url.startswith("http://") or url.startswith("https://")):
        return False, "URL অবশ্যই http:// অথবা https:// দিয়ে শুরু হতে হবে।"

    parsed = urlparse(url)
    if not parsed.netloc or "." not in parsed.netloc:
        return False, "URL-এর ডোমেইন অংশটি সঠিক মনে হচ্ছে না।"

    # চাইলে অতিরিক্ত চেক দিতে পারো (যেমন news site কিনা ইত্যাদি)
    return True, ""


In [64]:
def run_news_summarizer(language: str = "bn"):
    # 1) User input the URL
    url = input("📝 একটি news URL দিন: ").strip()

    # 2) Validate the URL
    is_ok, msg = validate_url(url)
    if not is_ok:
        print("❌ Invalid URL:", msg)
        return

    # 3) Try summarizing
    try:
        title, summary = summarize_news_url(url, language=language)

        print("\n===== 📰 ARTICLE TITLE =====")
        print(title)

        print("\n===== ✨ GENERATED SUMMARY =====\n")
        print(summary)
    except Exception as e:
        print("⚠️ কোনো একটা সমস্যা হয়েছে:", e)

# এখন শুধু এই লাইনটা চালালেই হবে:
run_news_summarizer(language="bn")   # English summary চাইলে language="en"


📝 একটি news URL দিন:  https://news.mit.edu/2025/study-suggests-40hz-sensory-stimulation-may-benefit-some-alzheimers-patients-1114


🌐 Fetching article...
📝 Extracted article length: 7201
🤖 Sending to Gemini 2.5-flash...

===== 📰 ARTICLE TITLE =====
Study suggests 40Hz sensory stimulation may benefit some Alzheimer’s patients for years | MIT News | Massachusetts Institute of Technology

===== ✨ GENERATED SUMMARY =====

একজন AI News Summarizer হিসেবে, নিচে দেওয়া আর্টিকেলটির ৫-৭টি পয়েন্টে একটি সহজ বাংলা সারাংশ দেওয়া হলো:

**মূল ঘটনা:** এমআইটি (MIT)-এর গবেষকরা আলঝেইমার্স রোগের চিকিৎসায় ৪০ হার্টজ (Hz) আলো ও শব্দ উদ্দীপনার দীর্ঘমেয়াদী প্রভাব নিয়ে একটি নতুন গবেষণা প্রকাশ করেছেন।

**সারাংশ:**

১.  **গবেষণার বিষয় ও স্থান:** এমআইটি (Massachusetts Institute of Technology)-এর গবেষকরা আলঝেইমার্স রোগের চিকিৎসায় ৪০ হার্টজ (Hz) আলো ও শব্দ উদ্দীপনার (GENUS পদ্ধতি) দীর্ঘমেয়াদী প্রভাব নিয়ে একটি পাইলট গবেষণা প্রকাশ করেছেন।
২.  **সময় ও অংশগ্রহণকারী:** এই গবেষণায় পাঁচজন স্বেচ্ছাসেবীকে একটি প্রাথমিক ক্লিনিকাল স্টাডির পর প্রায় দুই বছর ধরে তাদের বাড়িতে দৈনিক ৪০ হার্টজ আলো ও শব্দ উদ্দীপনা দেওয়া হয়েছে। এটি এই নিরাপদ ও নন-ইনভেসিভ চিকিৎসা পদ্ধ